In [1]:
import sys
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\simcore-sdk\src")
sys.path.append(r"C:\Users\anderegg\Documents\dev\OSPARC\osparc-simcore-ssh\packages\s3wrapper\src")

import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
import os
os.environ["SIMCORE_NODE_UUID"] = "3a97f542-93c4-419c-b5c9-bcf9ff3ada7e"
os.environ["PIPELINE_NODE_ID"] = "42"

os.environ["S3_ENDPOINT"] = "localhost:9001"
os.environ["S3_ACCESS_KEY"] = "12345678"
os.environ["S3_SECRET_KEY"] = "12345678"
os.environ["S3_BUCKET_NAME"] = "simcore"


In [3]:
from simcore_sdk.nodeports import PORTS

DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read True and auto write True
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75F49CF8>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'link.dd329e10-a906-42da-a7b3-4c4fec4a786f.out_1', 'timestamp': '2018-05-23T15:34:53.511Z'}
DEBUG:simcore_sdk.nodeports._item:Creating new data it

In [4]:
print(len(PORTS.inputs))
print(PORTS.inputs[0])

DEBUG:simcore_sdk.nodeports.nodeports:Getting inputs with autoread: True
DEBUG:simcore_sdk.nodeports.nodeports:Updating json configuration
DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read False and auto write False
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75F6EC18>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'link

DEBUG:simcore_sdk.nodeports._itemslist:Creating DataItemsList with [DataItem(key='in_1', label='File-url', desc='File-url', type='fileUrl', value='link.dd329e10-a906-42da-a7b3-4c4fec4a786f.out_1', timestamp='2018-05-23T15:34:53.511Z'), DataItem(key='in_2', label='Number', desc='Number', type='integer', value='link.dd329e10-a906-42da-a7b3-4c4fec4a786f.out_2', timestamp='2018-05-23T15:34:53.511Z')]
DEBUG:simcore_sdk.nodeports._itemslist:Creating DataItemsList with [DataItem(key='out_1', label='File-url', desc='File-url', type='fileUrl', value='Hey this still works or NOTTTTTT crazy shit man!!', timestamp='2018-06-19T10:31:14.728827'), DataItem(key='out_2', label='Number', desc='Number', type='integer', value=5, timestamp='2018-05-23T15:34:53.511Z')]
DEBUG:simcore_sdk.nodeports.nodeports:Initialising Nodeports object with version 0.1, inputs <simcore_sdk.nodeports._itemslist.DataItemsList object at 0x0000027F75F7E6A0> and outputs <simcore_sdk.nodeports._itemslist.DataItemsList object at 0

2
DataItem(key='in_1', label='File-url', desc='File-url', type='fileUrl', value='link.dd329e10-a906-42da-a7b3-4c4fec4a786f.out_1', timestamp='2018-05-23T15:34:53.511Z')


In [5]:
print(PORTS.inputs[0].get())


DEBUG:simcore_sdk.nodeports.nodeports:Getting inputs with autoread: True
DEBUG:simcore_sdk.nodeports.nodeports:Updating json configuration
DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read False and auto write False
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75F7E7B8>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'link

C:\Users\anderegg\AppData\Local\Temp\simcorefiles\in_1\in_1.simcore


In [6]:
print(PORTS.outputs[0])

DEBUG:simcore_sdk.nodeports.nodeports:Getting outputs with autoread: True
DEBUG:simcore_sdk.nodeports.nodeports:Updating json configuration
DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read False and auto write False
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75FB8240>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'lin

DataItem(key='out_1', label='File-url', desc='File-url', type='fileUrl', value='Hey this still works or NOTTTTTT crazy shit man!!', timestamp='2018-06-19T10:31:14.728827')


In [7]:
PORTS.outputs["out_1"].get()

DEBUG:simcore_sdk.nodeports.nodeports:Getting outputs with autoread: True
DEBUG:simcore_sdk.nodeports.nodeports:Updating json configuration
DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read False and auto write False
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75FBDC88>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'lin

'Hey this still works or NOTTTTTT crazy shit man!!'

In [8]:
PORTS.outputs["out_1"].set("Hey this still works or NOTTTTTT crazy shit man!!")

DEBUG:simcore_sdk.nodeports.nodeports:Getting outputs with autoread: True
DEBUG:simcore_sdk.nodeports.nodeports:Updating json configuration
DEBUG:simcore_sdk.nodeports.serialization:Creating Nodeports object with io object: <simcore_sdk.nodeports.io.IO object at 0x0000027F5D5DBF28>, auto read False and auto write False
DEBUG:simcore_sdk.nodeports.io:Getting ports configuration using Location.DATABASE
DEBUG:simcore_sdk.nodeports.io:Reading from database
DEBUG:simcore_sdk.nodeports.io:Reading from database for pipeline id 42 and node id 3a97f542-93c4-419c-b5c9-bcf9ff3ada7e
DEBUG:simcore_sdk.nodeports.serialization:Encoding object: <simcore_sdk.models.pipeline_models.ComputationalTask object at 0x0000027F75FD5A20>
DEBUG:simcore_sdk.nodeports.serialization:Encoding Node object
DEBUG:simcore_sdk.nodeports.io:Found and converted to json
DEBUG:simcore_sdk.nodeports.serialization:Decoding Data items json: {'key': 'in_1', 'label': 'File-url', 'desc': 'File-url', 'type': 'fileUrl', 'value': 'lin